In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import re
import sys
local_modules_path = (Path(".") / '..').resolve()
sys.path.append(str(local_modules_path))
from utils import merge_intervals

In [27]:
files = list(Path(".").glob("*.gz"))
files

[PosixPath('GSM5835477_ChIP_H3K27ac_HH18_PT_rep1.narrowPeak.gz'),
 PosixPath('GSM6152758_ATAC_HH35_DS_rep1.narrowPeak.gz'),
 PosixPath('GSM6152753_ATAC_E125_FB_rep2.narrowPeak.gz'),
 PosixPath('GSM5835479_ChIP_H3K27ac_HH28_DFL_rep1.narrowPeak.gz'),
 PosixPath('GSM6152759_ATAC_HH35_DS_rep2.narrowPeak.gz'),
 PosixPath('GSM5835478_ChIP_H3K27ac_HH18_PT_rep2.narrowPeak.gz'),
 PosixPath('GSM5835470_ChIP_H3K27ac_E125_DFL_rep1.narrowPeak.gz'),
 PosixPath('GSM6152750_ATAC_E95_PT_rep1.narrowPeak.gz'),
 PosixPath('GSM5835471_ChIP_H3K27ac_E125_FB_rep1.narrowPeak.gz'),
 PosixPath('GSM6152752_ATAC_E125_FB_rep1.narrowPeak.gz'),
 PosixPath('GSM6152755_ATAC_E125_WP_rep2.narrowPeak.gz'),
 PosixPath('GSM6152754_ATAC_E125_WP_rep1.narrowPeak.gz'),
 PosixPath('GSM5835476_ChIP_H3K27ac_E85_PT_rep2.narrowPeak.gz'),
 PosixPath('GSM5835482_ChIP_H3K27ac_HH35_DS_rep2.narrowPeak.gz'),
 PosixPath('GSM5835481_ChIP_H3K27ac_HH35_DS_rep1.narrowPeak.gz'),
 PosixPath('GSM6152757_ATAC_HH19_PT_rep2.narrowPeak.gz'),
 PosixPa

In [28]:
def get_meta(m): 
    return [ m.group(i) for i in [1,2,3,4] ]

matches = [ 
    get_meta(re.search(r"([A-Za-z0-9]+)_([A-Z]+)([0-9]+)_([A-Z]+)_rep[1-2].narrowPeak.gz", str(f)))
    for f in files ]
sample_meta = pd.DataFrame(matches, columns = ["assay", "species", "stage", "tissue"])
sample_meta["file"] = files

mapping_dict = {
    "PT" : "posterior_trunk", 
    "PFL" : "proximal_forelimb",
    "DFL" : "distal_forelimb",
    "FB" : "forebrain",
    "WP" : "whisker_pad",
    "DS" : "dorsal_skin"
}
sample_meta["tissue"] = sample_meta["tissue"].apply(lambda x: mapping_dict.get(x, x)) 

sample_meta["species"] = sample_meta["species"].map({
    "HH" : "gallus_gallus", 
    "E" : "mus_musculus"
})
sample_meta

,assay,species,stage,tissue,file
0,H3K27ac,gallus_gallus,18,posterior_trunk,GSM5835477_ChIP_H3K27ac_HH18_PT_rep1.narrowPea...
1,ATAC,gallus_gallus,35,dorsal_skin,GSM6152758_ATAC_HH35_DS_rep1.narrowPeak.gz
2,ATAC,mus_musculus,125,forebrain,GSM6152753_ATAC_E125_FB_rep2.narrowPeak.gz
3,H3K27ac,gallus_gallus,28,distal_forelimb,GSM5835479_ChIP_H3K27ac_HH28_DFL_rep1.narrowPe...
4,ATAC,gallus_gallus,35,dorsal_skin,GSM6152759_ATAC_HH35_DS_rep2.narrowPeak.gz
5,H3K27ac,gallus_gallus,18,posterior_trunk,GSM5835478_ChIP_H3K27ac_HH18_PT_rep2.narrowPea...
6,H3K27ac,mus_musculus,125,distal_forelimb,GSM5835470_ChIP_H3K27ac_E125_DFL_rep1.narrowPe...
7,ATAC,mus_musculus,95,posterior_trunk,GSM6152750_ATAC_E95_PT_rep1.narrowPeak.gz
8,H3K27ac,mus_musculus,125,forebrain,GSM5835471_ChIP_H3K27ac_E125_FB_rep1.narrowPea...
9,ATAC,mus_musculus,125,forebrain,GSM6152752_ATAC_E125_FB_rep1.narrowPeak.gz


In [29]:
unique = sample_meta[ ["assay", "tissue", "stage", "species"] ].drop_duplicates()
unique.shape

(16, 4)

In [30]:
import shutil
import time

column_names = [
    'chrom', 'start', 'end', 'name', 'score', 'strand',
    'signalValue', 'pValue', 'qValue', 'peak'
]
outdir = Path("merged")
outdir.mkdir(parents=True, exist_ok=True)

og_files = []
out_files = []
for i in range(unique.shape[0]):
    start_time = time.time()
    tissue = unique["tissue"].iloc[i]
    assay = unique["assay"].iloc[i]
    species = unique["species"].iloc[i]
    stage = unique["stage"].iloc[i]
    
    files = sample_meta.loc[ 
        (sample_meta["tissue"] == tissue) & 
        (sample_meta["assay"] == assay) & 
        (sample_meta["stage"] == stage) &
        (sample_meta["species"] == species), 
        "file"]
    outfile = f"{tissue}_{stage}_{assay}_{species}.bed.gz"
    df_all = pd.concat(
                [ pd.read_csv(g, sep='\t', header=None, names=column_names) 
                 for g in files ],
                axis=0)
    merged = merge_intervals(df_all)
    merged.to_csv(outdir / outfile, sep = "\t", index = False, header = False)
    out_files.append(outfile)
    og_files.append(",".join(files.astype(str)))
    print(f"{tissue} {assay} {species} done, took {time.time()-start_time}")

posterior_trunk H3K27ac gallus_gallus done, took 10.836852312088013
dorsal_skin ATAC gallus_gallus done, took 23.11153531074524
forebrain ATAC mus_musculus done, took 29.122607469558716
distal_forelimb H3K27ac gallus_gallus done, took 5.106316566467285
distal_forelimb H3K27ac mus_musculus done, took 5.064899921417236
posterior_trunk ATAC mus_musculus done, took 23.96013903617859
forebrain H3K27ac mus_musculus done, took 5.074098348617554
whisker_pad ATAC mus_musculus done, took 30.63923668861389
posterior_trunk H3K27ac mus_musculus done, took 13.5990629196167
dorsal_skin H3K27ac gallus_gallus done, took 2.4971776008605957
posterior_trunk ATAC gallus_gallus done, took 29.184200763702393
proximal_forelimb H3K27ac gallus_gallus done, took 4.653214931488037
forebrain ATAC gallus_gallus done, took 29.317896127700806
proximal_forelimb H3K27ac mus_musculus done, took 6.040230751037598
forebrain H3K27ac gallus_gallus done, took 0.9347119331359863
whisker_pad H3K27ac mus_musculus done, took 7.4

In [31]:
unique["filename"] = out_files
unique["original"] = og_files
unique

,assay,tissue,stage,species,filename,original
0,H3K27ac,posterior_trunk,18,gallus_gallus,posterior_trunk_18_H3K27ac_gallus_gallus.bed.gz,GSM5835477_ChIP_H3K27ac_HH18_PT_rep1.narrowPea...
1,ATAC,dorsal_skin,35,gallus_gallus,dorsal_skin_35_ATAC_gallus_gallus.bed.gz,"GSM6152758_ATAC_HH35_DS_rep1.narrowPeak.gz,GSM..."
2,ATAC,forebrain,125,mus_musculus,forebrain_125_ATAC_mus_musculus.bed.gz,"GSM6152753_ATAC_E125_FB_rep2.narrowPeak.gz,GSM..."
3,H3K27ac,distal_forelimb,28,gallus_gallus,distal_forelimb_28_H3K27ac_gallus_gallus.bed.gz,GSM5835479_ChIP_H3K27ac_HH28_DFL_rep1.narrowPe...
6,H3K27ac,distal_forelimb,125,mus_musculus,distal_forelimb_125_H3K27ac_mus_musculus.bed.gz,GSM5835470_ChIP_H3K27ac_E125_DFL_rep1.narrowPe...
7,ATAC,posterior_trunk,95,mus_musculus,posterior_trunk_95_ATAC_mus_musculus.bed.gz,"GSM6152750_ATAC_E95_PT_rep1.narrowPeak.gz,GSM6..."
8,H3K27ac,forebrain,125,mus_musculus,forebrain_125_H3K27ac_mus_musculus.bed.gz,GSM5835471_ChIP_H3K27ac_E125_FB_rep1.narrowPea...
10,ATAC,whisker_pad,125,mus_musculus,whisker_pad_125_ATAC_mus_musculus.bed.gz,"GSM6152755_ATAC_E125_WP_rep2.narrowPeak.gz,GSM..."
12,H3K27ac,posterior_trunk,85,mus_musculus,posterior_trunk_85_H3K27ac_mus_musculus.bed.gz,GSM5835476_ChIP_H3K27ac_E85_PT_rep2.narrowPeak...
13,H3K27ac,dorsal_skin,35,gallus_gallus,dorsal_skin_35_H3K27ac_gallus_gallus.bed.gz,GSM5835482_ChIP_H3K27ac_HH35_DS_rep2.narrowPea...


In [32]:
unique["tissue"] = unique["tissue"].astype(str) + "_" + unique["stage"].astype(str)
unique

,assay,tissue,stage,species,filename,original
0,H3K27ac,posterior_trunk_18,18,gallus_gallus,posterior_trunk_18_H3K27ac_gallus_gallus.bed.gz,GSM5835477_ChIP_H3K27ac_HH18_PT_rep1.narrowPea...
1,ATAC,dorsal_skin_35,35,gallus_gallus,dorsal_skin_35_ATAC_gallus_gallus.bed.gz,"GSM6152758_ATAC_HH35_DS_rep1.narrowPeak.gz,GSM..."
2,ATAC,forebrain_125,125,mus_musculus,forebrain_125_ATAC_mus_musculus.bed.gz,"GSM6152753_ATAC_E125_FB_rep2.narrowPeak.gz,GSM..."
3,H3K27ac,distal_forelimb_28,28,gallus_gallus,distal_forelimb_28_H3K27ac_gallus_gallus.bed.gz,GSM5835479_ChIP_H3K27ac_HH28_DFL_rep1.narrowPe...
6,H3K27ac,distal_forelimb_125,125,mus_musculus,distal_forelimb_125_H3K27ac_mus_musculus.bed.gz,GSM5835470_ChIP_H3K27ac_E125_DFL_rep1.narrowPe...
7,ATAC,posterior_trunk_95,95,mus_musculus,posterior_trunk_95_ATAC_mus_musculus.bed.gz,"GSM6152750_ATAC_E95_PT_rep1.narrowPeak.gz,GSM6..."
8,H3K27ac,forebrain_125,125,mus_musculus,forebrain_125_H3K27ac_mus_musculus.bed.gz,GSM5835471_ChIP_H3K27ac_E125_FB_rep1.narrowPea...
10,ATAC,whisker_pad_125,125,mus_musculus,whisker_pad_125_ATAC_mus_musculus.bed.gz,"GSM6152755_ATAC_E125_WP_rep2.narrowPeak.gz,GSM..."
12,H3K27ac,posterior_trunk_85,85,mus_musculus,posterior_trunk_85_H3K27ac_mus_musculus.bed.gz,GSM5835476_ChIP_H3K27ac_E85_PT_rep2.narrowPeak...
13,H3K27ac,dorsal_skin_35,35,gallus_gallus,dorsal_skin_35_H3K27ac_gallus_gallus.bed.gz,GSM5835482_ChIP_H3K27ac_HH35_DS_rep2.narrowPea...


In [33]:
meta = unique[["tissue","assay","species","filename","original"]]
meta.to_csv("meta.tsv", sep="\t", index=False)